In [ ]:
#!/usr/bin/env python
# coding: utf-8
import openai
import urllib.request
from unidiff import PatchSet
from github import Github
import tiktoken
from collections import namedtuple
from pprint import pprint

REPO_NAME = 'Uniswap/interface'
openai.api_key = "I don't think so"
GITHUB_ACCESS_TOKEN = 'no chance'
MSG_PROMPT = '''You are an experienced software engineer who writes clean, well-commented, and concise code.
You are given a PR with a changelog and a diff. You are asked to review the PR and provide feedback. Summarize your response.
'''
PROMPT = [
    {"role": "user", "content": MSG_PROMPT},
]


def getRepo(github_access_token=GITHUB_ACCESS_TOKEN):
    github = Github(github_access_token)
    return github.get_repo(REPO_NAME)


def getPrs(repo=getRepo()):
    return repo.get_pulls(base='main', state='open')


def formatPrompt(in_prompt, changes, title, description):
    prompt = in_prompt.copy()
    prompt.append({"role": "user", "content": f'PR title:\n{title}'})
    prompt.append(
        {"role": "user", "content": f'PR description:\n{description}'})
    for change in changes:
        prompt.append(
            {"role": "user", "content": f'File contents:\n{change.file}'})
        prompt.append({"role": "user", "content": f'Change:\n{change.patch}'})
    return prompt


def getReview(changes, pr):
    final_prompt = formatPrompt(PROMPT, changes, pr.title, pr.body)
    # print(final_prompt)
    intial_reply = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=final_prompt
    )

    '''reply_prompt = {"role": "assistant", "content": f"{intial_reply['choices'][0]['message']['content']}"}
        summarize_prompt = {"role": "user", "content": f"Summarize your last response."}
        reply_guy_prompt = final_prompt + [reply_prompt, summarize_prompt]
        print(reply_guy_prompt)
        return openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages = reply_guy_prompt
        )['choices'][0]['message']['content']'''
    return intial_reply['choices'][0]['message']['content']


def juryGetReview(n, f):
    results = []
    for _ in range(n):
        result = f()
        # print(result)
        results.append(result)

    msg_prompt = f'''You are an experienced software engineer who writes clean, well-commented, and concise code.
You are given a list of PR reviews written by other software engineers. Choose the best one to provide for a PR.
'''

    prompt = {"role": "user", "content": msg_prompt}
    reviews = [{"role": "user", "content": result} for result in results]
    return openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[prompt] + reviews
    )


repo = getRepo()
Change = namedtuple('Change', ['file', 'patch'])
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

for pr in getPrs(repo):
    print('PR title:', pr.title)

    diff = urllib.request.urlopen(pr.diff_url)
    encoding = diff.headers.get_charsets()[0]
    patch = PatchSet(diff, encoding=encoding)

    prompt_tokens = len(enc.encode(str(PROMPT)))
    total_tokens = prompt_tokens + 100  # idk, title and description and changes

    changes = []
    for change in patch.modified_files:
        url = repo.get_contents(change.path).download_url
        file_contents = urllib.request.urlopen(url).read().decode('utf-8')
        changes.append(Change(str(change), file_contents))

        changelog_tokens = len(enc.encode(str(change)))
        prompt_tokens = 100  # idk
        file_tokens = len(enc.encode(file_contents))
        change_tokens = changelog_tokens + prompt_tokens + file_tokens
        total_tokens += change_tokens

    if (total_tokens > 4096):
        print("Files too long. todo: summarize")
        continue

    # print(total_tokens)

    result = getReview(changes, pr)#juryGetReview(6)
    pprint(result)
